In [2]:
import ccxt
import pandas as pd
import numpy as np
import time

# -----------------------------
# Configuration
# -----------------------------

EXCHANGE_ID = 'binance'   # Example: Binance Spot
SYMBOL = 'BTC/USDT'       # Trading pair
TIMEFRAME = '1h'          # e.g., 1-hour candles
LIMIT = 1000              # Number of candles to fetch

INITIAL_CAPITAL = 10000.0
TRADE_SIZE = 0.001  # Adjust based on your capital and asset

# Parameter ranges for optimization
stop_loss_range = [0.005, 0.01, 0.015]     # e.g., 0.5%, 1%, 1.5%
take_profit_range = [0.01, 0.012, 0.015]   # e.g., 1%, 1.2%, 1.5%

# -----------------------------
# Fetching Data with CCXT
# -----------------------------

exchange = getattr(ccxt, EXCHANGE_ID)()
# If needed, set API keys:
# exchange.apiKey = "YOUR_API_KEY"
# exchange.secret = "YOUR_SECRET"

# Fetch OHLCV: returns [[timestamp, open, high, low, close, volume], ...]
ohlcv = exchange.fetch_ohlcv(SYMBOL, timeframe=TIMEFRAME, limit=LIMIT)

# Convert to DataFrame
data = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
data.sort_values('timestamp', inplace=True)
data.reset_index(drop=True, inplace=True)

# -----------------------------
# Backtesting Function
# -----------------------------

def backtest_strategy(data, stop_loss_percent, take_profit_percent):
    """
    Backtest a strategy where we:
    - Periodically open both a long and short position.
    - Close positions when SL or TP is hit.
    This is a simplified example for demonstration.
    """

    positions = []
    closed_positions = []
    capital = INITIAL_CAPITAL

    # In a real scenario, define your actual entry criteria
    for i in range(len(data)):
        close_price = data['close'].iloc[i]

        # Example condition: Open positions every 500 candles (this is arbitrary)
        if (i % 500 == 0) and i < (len(data) - 1):
            long_entry_price = close_price
            short_entry_price = close_price

            long_sl = long_entry_price * (1 - stop_loss_percent)
            long_tp = long_entry_price * (1 + take_profit_percent)

            short_sl = short_entry_price * (1 + stop_loss_percent)
            short_tp = short_entry_price * (1 - take_profit_percent)

            positions.append({
                'side': 'long',
                'entry_price': long_entry_price,
                'stop_loss': long_sl,
                'take_profit': long_tp,
                'open_index': i,
                'closed': False
            })
            positions.append({
                'side': 'short',
                'entry_price': short_entry_price,
                'stop_loss': short_sl,
                'take_profit': short_tp,
                'open_index': i,
                'closed': False
            })

        # Check SL/TP for open positions
        for pos in positions:
            if not pos['closed']:
                if pos['side'] == 'long':
                    # Long position SL/TP check
                    if data['low'].iloc[i] <= pos['stop_loss']:
                        # SL hit
                        exit_price = pos['stop_loss']
                        profit = (exit_price - pos['entry_price']) * (TRADE_SIZE * (capital / data['close'].iloc[pos['open_index']]))
                        pos['closed'] = True
                        pos['exit_price'] = exit_price
                        pos['exit_index'] = i
                        pos['profit'] = profit
                        closed_positions.append(pos)
                        capital += profit
                    elif data['high'].iloc[i] >= pos['take_profit']:
                        # TP hit
                        exit_price = pos['take_profit']
                        profit = (exit_price - pos['entry_price']) * (TRADE_SIZE * (capital / data['close'].iloc[pos['open_index']]))
                        pos['closed'] = True
                        pos['exit_price'] = exit_price
                        pos['exit_index'] = i
                        pos['profit'] = profit
                        closed_positions.append(pos)
                        capital += profit

                elif pos['side'] == 'short':
                    # Short position SL/TP check
                    if data['high'].iloc[i] >= pos['stop_loss']:
                        # SL hit for short
                        exit_price = pos['stop_loss']
                        profit = (pos['entry_price'] - exit_price) * (TRADE_SIZE * (capital / data['close'].iloc[pos['open_index']]))
                        pos['closed'] = True
                        pos['exit_price'] = exit_price
                        pos['exit_index'] = i
                        pos['profit'] = profit
                        closed_positions.append(pos)
                        capital += profit
                    elif data['low'].iloc[i] <= pos['take_profit']:
                        # TP hit for short
                        exit_price = pos['take_profit']
                        profit = (pos['entry_price'] - exit_price) * (TRADE_SIZE * (capital / data['close'].iloc[pos['open_index']]))
                        pos['closed'] = True
                        pos['exit_price'] = exit_price
                        pos['exit_index'] = i
                        pos['profit'] = profit
                        closed_positions.append(pos)
                        capital += profit

    # Calculate performance metrics
    total_trades = len(closed_positions)
    if total_trades > 0:
        wins = [p for p in closed_positions if p['profit'] > 0]
        losses = [p for p in closed_positions if p['profit'] <= 0]

        total_profit = sum(p['profit'] for p in closed_positions)
        win_rate = len(wins) / total_trades if total_trades > 0 else 0
        avg_win = np.mean([p['profit'] for p in wins]) if wins else 0
        avg_loss = np.mean([p['profit'] for p in losses]) if losses else 0
    else:
        total_profit = 0
        win_rate = 0
        avg_win = 0
        avg_loss = 0

    return {
        'stop_loss_percent': stop_loss_percent,
        'take_profit_percent': take_profit_percent,
        'final_capital': capital,
        'total_trades': total_trades,
        'total_profit': total_profit,
        'win_rate': win_rate,
        'avg_win': avg_win,
        'avg_loss': avg_loss
    }

# -----------------------------
# Parameter Optimization
# -----------------------------

results = []
for sl in stop_loss_range:
    for tp in take_profit_range:
        result = backtest_strategy(data, stop_loss_percent=sl, take_profit_percent=tp)
        results.append(result)

# Convert results to DataFrame and analyze
results_df = pd.DataFrame(results)
print("All Results:")
print(results_df.sort_values('total_profit', ascending=False))

best_params = results_df.iloc[results_df['total_profit'].argmax()]
print("\nBest parameters found:")
print(best_params)


All Results:
   stop_loss_percent  take_profit_percent  final_capital  total_trades  \
5              0.010                0.015   10000.099997             4   
4              0.010                0.012   10000.039998             4   
2              0.005                0.015    9999.999999             4   
3              0.010                0.010    9999.999998             4   
8              0.015                0.015    9999.999995             4   
1              0.005                0.012    9999.969999             4   
0              0.005                0.010    9999.949999             4   
7              0.015                0.012    9999.939996             4   
6              0.015                0.010    9999.899997             4   

   total_profit  win_rate   avg_win  avg_loss  
5      0.099997      0.50  0.149999 -0.100000  
4      0.039998      0.50  0.120000 -0.100001  
2     -0.000001      0.25  0.149998 -0.050000  
3     -0.000002      0.50  0.099999 -0.100000  
8     